In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import random
from matplotlib import pyplot as plt

/home/mark/Desktop/py3.7/cv/paddleOCR/lib/python3.7/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
os.getcwd()

'/home/mark/Desktop/py3.7/cv/paddleOCR/code/ai_cup_f/preprocessing'

In [3]:
#directory must contain public foler
dataset_path = "/home/mark/Desktop/py3.7/cv/datasets/ai_cup_f/"
img_path=dataset_path+"train/img/"
#where you save img
out_path= dataset_path+"/train_str_cut/img/"

In [4]:
def crop_polygon(img,points):
    #crop img with given points([[x1,y1],[x2,y2]...])
    #require cv2, np
    pts = np.array(points)
    rect = cv2.boundingRect(pts)
    x,y,w,h = rect
    croped_img = img[y:y+h, x:x+w].copy()
    pts = pts - pts.min(axis=0)
    mask = np.zeros(croped_img.shape[:2], np.uint8)
    cv2.drawContours(mask, [pts], -1, (255, 255, 255), -1, cv2.LINE_AA)
    dst = cv2.bitwise_and(croped_img, croped_img, mask=mask)
    return dst

In [5]:
df = pd.read_csv(dataset_path+"/train_ids_cor.csv",encoding="utf8")
df.replace(-1,0,inplace=True)#turn negative to 0
df.head()

,imgs,ids,pt1,pt2,pt3,pt4,pt5,pt6,pt7,pt8,labels
0,img_7818,1,267,1107,354,1104,347,1202,258,1203,多
1,img_7818,1,359,1105,442,1100,433,1197,350,1200,多
2,img_7818,1,443,1102,530,1096,525,1194,434,1199,選
3,img_7818,1,529,1098,620,1093,617,1192,524,1194,物
4,img_7818,1,624,1094,714,1090,711,1192,618,1190,販


In [6]:
#save pic
opened_img = None
img = None
pic_count=0
cut_img=[]
for index, row in df.iterrows():
    current_img = df.imgs[index]#get file name
    #get position data
    points = [[row["pt"+str(x)],row["pt"+str(x+1)]] for x in range(1,9,2)]#loop thru 1,2~7,8
    #turn points into np.array
    points = np.array(points, np.int32)
    #open pic
    pic_count+=1
    if current_img != opened_img:
        img = cv2.imread(img_path+current_img+".jpg")
        opened_img = current_img
        pic_count=0
    #crop pic
    cropped_img = crop_polygon(img,points)
    #save pic
    cv2.imwrite(out_path+opened_img+"_"+str(pic_count)+".jpg",cropped_img)
    cut_img.append(opened_img+"_"+str(pic_count)+".jpg")

In [7]:
cut_img[:10]

['img_7818_0.jpg',
 'img_7818_1.jpg',
 'img_7818_2.jpg',
 'img_7818_3.jpg',
 'img_7818_4.jpg',
 'img_7818_5.jpg',
 'img_7818_6.jpg',
 'img_7818_7.jpg',
 'img_7818_8.jpg',
 'img_7818_9.jpg']

In [8]:
df.imgs = cut_img
df.head()

,imgs,ids,pt1,pt2,pt3,pt4,pt5,pt6,pt7,pt8,labels
0,img_7818_0.jpg,1,267,1107,354,1104,347,1202,258,1203,多
1,img_7818_1.jpg,1,359,1105,442,1100,433,1197,350,1200,多
2,img_7818_2.jpg,1,443,1102,530,1096,525,1194,434,1199,選
3,img_7818_3.jpg,1,529,1098,620,1093,617,1192,524,1194,物
4,img_7818_4.jpg,1,624,1094,714,1090,711,1192,618,1190,販


In [9]:
df.isna().sum()

imgs      0
ids       0
pt1       0
pt2       0
pt3       0
pt4       0
pt5       0
pt6       0
pt7       0
pt8       0
labels    4
dtype: int64

In [10]:
df.to_csv(dataset_path+"labels.csv",encoding="utf8",index=False)